In [29]:
# 📌 1. 기본 라이브러리
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from collections import Counter

In [30]:
# 📌 2. 데이터 불러오기
use_cols = [
    'ID',
    '_1순위카드이용건수',
    '_2순위카드이용금액',
    '입회일자_신용',
    '최종카드발급경과월',
    '탈회횟수_발급1년이내',
    '월상환론한도금액',
    '카드이용한도금액_B2M',
    'CA한도금액',
    '_1순위교통업종_이용금액',
    '_1순위업종_이용금액',
    '_2순위쇼핑업종_이용금액',
    '_2순위업종_이용금액',
    '_3순위쇼핑업종_이용금액',
    '_3순위업종_이용금액',
    '쇼핑_도소매_이용금액',
    '연속유실적개월수_기본_24M_카드',
    '연체입금원금_B0M',
    '연체입금원금_B2M',
    '연체입금원금_B5M',
    '이용개월수_할부_무이자_R3M',
    '이용건수_오프라인_R6M',
    '이용금액_선결제_B0M',
    '이용금액_오프라인_B0M',
    '이용금액_오프라인_R3M',
    '이용금액_오프라인_R6M',
    '이용금액_일시불_R12M',
    '이용금액_체크_R12M',
    '이용금액_할부_무이자_R12M',
    '이용금액_A페이_R3M',
    '이용금액대',
    '정상입금원금_B2M',
    '정상입금원금_B5M',
    '최대이용금액_일시불_R12M',
    '최대이용금액_체크_R12M',
    '최종이용일자_CA_년',
    '할부금액_무이자_3M_R12M',
    '할인금액_청구서_B0M',
    '청구금액_B0',
    '청구금액_R6M',
    '월중평잔_일시불',
    '잔액_일시불_B0M',
    '평잔_일시불_6M',
    '평잔_CA_해외_3M',
    '컨택건수_보험_TM_B0M',
    '컨택건수_카드론_EM_R6M',
    '여유_숙박이용금액',
    '여유_항공이용금액',
    '_3순위여유업종',
    '_3순위여유업종_이용금액',
    'RP건수_보험_B0M',
    '이용금액_D페이_R3M',
    '이용건수_선결제_B0M',
    '건수_할부전환_R6M',
    'Segment'
    ]

In [31]:
# 👇 1백만 행 단위로 나눠서 불러오기
chunks = pd.read_csv(
    '병합/merged_final_train_셀_추가병합.csv',
    usecols=use_cols,
    chunksize=1_000_000
)

# 👇 불러온 청크를 병합 (메모리가 허용하는 선에서)
df_list = []
for i, chunk in enumerate(chunks):
    print(f"✅ Chunk {i+1} loaded: {chunk.shape}")
    df_list.append(chunk)

train_partial = pd.concat(df_list, ignore_index=True)
print(f"\n📦 최종 데이터 shape: {train_partial.shape}")

✅ Chunk 1 loaded: (1000000, 55)
✅ Chunk 2 loaded: (1000000, 55)
✅ Chunk 3 loaded: (400000, 55)

📦 최종 데이터 shape: (2400000, 55)


In [33]:
train_partial.to_csv('병합/53_train.csv', index=False, encoding='utf-8-sig')

In [25]:
use_cols_test = [col for col in use_cols if col != 'Segment']

In [26]:
test = pd.read_csv('병합/merged_final_test_셀_추가병합.csv', usecols=use_cols_test)

In [27]:
test.to_csv('병합/53_test.csv', index=False, encoding='utf-8-sig')

In [9]:
train_partial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 55 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   ID                  object
 1   입회일자_신용             int64 
 2   탈회횟수_발급1년이내         int64 
 3   _1순위카드이용건수          int64 
 4   _2순위카드이용금액          int64 
 5   최종카드발급경과월           int64 
 6   CA한도금액              int64 
 7   월상환론한도금액            int64 
 8   카드이용한도금액_B2M        int64 
 9   이용금액_일시불_R12M       int64 
 10  이용금액_할부_무이자_R12M    int64 
 11  이용금액_체크_R12M        int64 
 12  최대이용금액_일시불_R12M     int64 
 13  최대이용금액_체크_R12M      int64 
 14  이용개월수_할부_무이자_R3M    int64 
 15  쇼핑_도소매_이용금액         int64 
 16  _1순위업종_이용금액         int64 
 17  _2순위업종_이용금액         int64 
 18  _3순위업종_이용금액         int64 
 19  _2순위쇼핑업종_이용금액       int64 
 20  _3순위쇼핑업종_이용금액       int64 
 21  _1순위교통업종_이용금액       int64 
 22  할부금액_무이자_3M_R12M    int64 
 23  이용금액_오프라인_R6M       int64 
 24  이용건수_오프라인_R6M       int64 
 25  이용금액_오프라인_R3M     

In [12]:
train = train_partial

In [28]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 54 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ID                  600000 non-null  object
 1   입회일자_신용             600000 non-null  int64 
 2   탈회횟수_발급1년이내         600000 non-null  int64 
 3   _1순위카드이용건수          600000 non-null  int64 
 4   _2순위카드이용금액          600000 non-null  int64 
 5   최종카드발급경과월           600000 non-null  int64 
 6   CA한도금액              600000 non-null  int64 
 7   월상환론한도금액            600000 non-null  int64 
 8   카드이용한도금액_B2M        600000 non-null  int64 
 9   이용금액_일시불_R12M       600000 non-null  int64 
 10  이용금액_할부_무이자_R12M    600000 non-null  int64 
 11  이용금액_체크_R12M        600000 non-null  int64 
 12  최대이용금액_일시불_R12M     600000 non-null  int64 
 13  최대이용금액_체크_R12M      600000 non-null  int64 
 14  이용개월수_할부_무이자_R3M    600000 non-null  int64 
 15  쇼핑_도소매_이용금액         600000 non-null  int64 
 16  _1

In [13]:
# ▶ 1. 피처(X), 정답(y) 분리
X_vif = train.drop(columns=['Segment'])
y_vif = train['Segment']

# ▶ 2. 필요 시 레이블 인코딩 (CatBoost는 문자형도 처리 가능하지만 교차검증 편의 위해 사용)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_vif_encoded = le.fit_transform(y_vif)

In [14]:
# 'ID' 컬럼 제거
X_vif = X_vif.drop(columns=['ID'], errors='ignore')

In [15]:
X_vif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 53 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   입회일자_신용             int64 
 1   탈회횟수_발급1년이내         int64 
 2   _1순위카드이용건수          int64 
 3   _2순위카드이용금액          int64 
 4   최종카드발급경과월           int64 
 5   CA한도금액              int64 
 6   월상환론한도금액            int64 
 7   카드이용한도금액_B2M        int64 
 8   이용금액_일시불_R12M       int64 
 9   이용금액_할부_무이자_R12M    int64 
 10  이용금액_체크_R12M        int64 
 11  최대이용금액_일시불_R12M     int64 
 12  최대이용금액_체크_R12M      int64 
 13  이용개월수_할부_무이자_R3M    int64 
 14  쇼핑_도소매_이용금액         int64 
 15  _1순위업종_이용금액         int64 
 16  _2순위업종_이용금액         int64 
 17  _3순위업종_이용금액         int64 
 18  _2순위쇼핑업종_이용금액       int64 
 19  _3순위쇼핑업종_이용금액       int64 
 20  _1순위교통업종_이용금액       int64 
 21  할부금액_무이자_3M_R12M    int64 
 22  이용금액_오프라인_R6M       int64 
 23  이용건수_오프라인_R6M       int64 
 24  이용금액_오프라인_R3M       int64 
 25  이용금액_오프라인_B0M     

In [16]:
# ▶ CatBoost 설치
!pip install -q catboost

In [19]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# ▶ 범주형 컬럼 자동 추출
cat_features = X_vif.select_dtypes(include=['object']).columns.tolist()

# ▶ 교차검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []

# ▶ CatBoost 파라미터
model_params = {
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "task_type": "GPU",
    "learning_rate": 0.01,
    "iterations": 70000,
    "l2_leaf_reg": 50,
    "random_seed": 42,
    "od_type": "Iter",
    "depth": 5,
    "early_stopping_rounds": 15000,
    "border_count": 64,
    "verbose": 1000
}

# ▶ 교차검증 반복
for fold, (train_idx, val_idx) in enumerate(cv.split(X_vif, y_vif_encoded)):
    print(f"▶ Fold {fold + 1}")

    X_train, X_val = X_vif.iloc[train_idx], X_vif.iloc[val_idx]
    y_train, y_val = y_vif_encoded[train_idx], y_vif_encoded[val_idx]

    train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
    val_pool = Pool(data=X_val, label=y_val, cat_features=cat_features)

    model = CatBoostClassifier(**model_params)
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)

    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average='micro')
    f1_scores.append(f1)

    print(f"✅ F1_micro: {f1:.5f}")

# ▶ 평균 F1 출력
print(f"\n🎯 평균 F1_micro (5 folds): {np.mean(f1_scores):.5f}")

▶ Fold 1
0:	learn: 1.5800633	test: 1.5800855	best: 1.5800855 (0)	total: 65.8ms	remaining: 1h 16m 46s
1000:	learn: 0.2856975	test: 0.2861223	best: 0.2861223 (1000)	total: 40.1s	remaining: 46m 3s
2000:	learn: 0.2674476	test: 0.2679800	best: 0.2679800 (2000)	total: 1m 18s	remaining: 44m 28s
3000:	learn: 0.2588661	test: 0.2596319	best: 0.2596319 (3000)	total: 1m 55s	remaining: 43m 3s
4000:	learn: 0.2531843	test: 0.2541970	best: 0.2541970 (4000)	total: 2m 33s	remaining: 42m 12s
5000:	learn: 0.2488102	test: 0.2500828	best: 0.2500828 (5000)	total: 3m 11s	remaining: 41m 30s
6000:	learn: 0.2454540	test: 0.2469787	best: 0.2469787 (6000)	total: 3m 49s	remaining: 40m 52s
7000:	learn: 0.2426396	test: 0.2444250	best: 0.2444250 (7000)	total: 4m 27s	remaining: 40m 10s
8000:	learn: 0.2403019	test: 0.2423349	best: 0.2423349 (8000)	total: 5m 6s	remaining: 39m 32s
9000:	learn: 0.2383079	test: 0.2405876	best: 0.2405876 (9000)	total: 5m 43s	remaining: 38m 48s
10000:	learn: 0.2365724	test: 0.2390705	best: 0.

In [20]:
# ▶ cat_features 정의 (범주형...없긴 함)
cat_features = X_vif.select_dtypes(include='object').columns.tolist()

In [22]:
import pickle
import pandas as pd
import numpy as np

# ▶ 저장 경로 지정
base_path = '피처_53'

# ▶ 1. Fold별 f1_micro 점수 저장
with open(f'{base_path}/f1_scores.pkl', 'wb') as f:
    pickle.dump(f1_scores, f)

# ▶ 2. 평균 f1_micro 점수 저장
with open(f'{base_path}/f1_mean.txt', 'w') as f:
    f.write(str(np.mean(f1_scores)))

# ▶ 3. LabelEncoder 저장 (Segment 인코딩 복원용)
with open(f'{base_path}/label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

# ▶ 4. 학습용 피처(X_vif)와 타깃(y_vif_encoded) 저장
X_vif.to_csv(f'{base_path}/X_vif.csv', index=False)
pd.Series(y_vif_encoded, name='Segment_encoded').to_csv(f'{base_path}/y_vif_encoded.csv', index=False)

# ▶ 5. 범주형 컬럼 리스트 저장
with open(f'{base_path}/cat_features.pkl', 'wb') as f:
    pickle.dump(cat_features, f)

# ▶ 6. 최종 모델 저장 (선택)
# final_model.save_model(f'{base_path}/final_catboost_model.cbm')
